# Week 1 Homework: Build the Fund's Data Pipeline

> *"Bad data is worse than no data. With no data, you know you're guessing. With bad data, you think you know."*

## The Mission

You've been hired as the first data engineer at a small quantitative fund. There are three portfolio managers and a handful of researchers. They all need data. Right now, everyone downloads their own CSVs, cleans them (or doesn't), and stores them on their laptops. One PM has an Excel sheet with prices going back to 2015 that he swears is "clean." Another researcher just discovered that her TSLA backtest included a phantom 80% crash on the day of the 2020 stock split. Nobody knows whose numbers to trust.

Your job: build the fund's data pipeline — the single source of truth that every analyst and every model will depend on. This means downloading from a vendor API, cleaning the inevitable mess, running automated quality checks, storing everything in a format that doesn't waste everyone's time, and making it easy to load exactly the slice of data someone needs. If the data is wrong, every model downstream is wrong. Every backtest is wrong. Every risk report is wrong.

This isn't a software engineering exercise dressed up as finance. It's the actual first task a new quant data engineer faces at firms like Two Sigma, Citadel, or DE Shaw — except they're doing it for hundreds of thousands of instruments across asset classes, and you're starting with 50 US equities. The pipeline you build tonight is the foundation for every homework in this course. You'll reuse it in Weeks 2 through 6 and extend it further in Week 13.

Four deliverables. One evening. Let's build something real.

## Deliverables

1. **`FinancialDataLoader` class** — The complete data lifecycle: download, clean, validate, store, and load. Handles 50+ tickers without crashing. This is the big build.

2. **Data quality report** — Per-ticker metrics (completeness, stale prices, OHLC violations) with letter grades. Universe-level summary. Which tickers are production-ready and which would get you fired?

3. **Storage format benchmark** — CSV vs. Parquet, pandas vs. Polars, at multiple scales. The numbers will tell you why the industry is migrating.

4. **Return statistics summary** — Mean, volatility, skewness, kurtosis, tail behavior for 10 representative tickers. This is your bridge to Week 2 — the numbers here will break every Gaussian assumption your ML models were built on.

In [ ]:
# ── Setup ──────────────────────────────────────────────────
import json
import time
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import polars as pl
import scipy.stats as scipy_stats
import yfinance as yf
from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay

plt.rcParams["figure.dpi"] = 120
plt.rcParams["axes.grid"] = True
plt.rcParams["grid.alpha"] = 0.3

CACHE_DIR = Path("code/.cache/hw")
CACHE_DIR.mkdir(exist_ok=True, parents=True)

---

## Deliverable 1: The `FinancialDataLoader` Class

This is the centerpiece. You're building a class that handles the complete data lifecycle — downloading from yfinance, cleaning the gaps and anomalies, running automated quality checks, saving to Parquet, and loading back with flexible filtering. The class needs to handle at least 50 tickers without crashing, even when individual tickers fail (and some will).

Think about what makes this different from a quick `yf.download()` call: error handling, quality scoring, format choice, metadata tracking. Every method should be defensive — flag problems, don't crash. A production pipeline that dies on one bad ticker out of 500 is useless.

**Requirements:**
- Download daily OHLCV for a configurable ticker list and date range
- Forward-fill missing days up to a configurable limit; flag longer gaps
- Compute per-ticker quality scores (completeness, OHLC consistency, stale prices)
- Save to Parquet (wide and long formats) with sidecar metadata JSON
- Load back with filtering by ticker, date range, and minimum quality grade

Step 4 is where it gets interesting — you'll discover that some "blue chip" stocks have surprisingly messy histories, and that the quality gap between large-cap liquid names and everything else is wider than you'd expect.

In [ ]:
# TODO: Build the FinancialDataLoader class
# Start with __init__ — what state does the loader need to track?
# Then implement: download(), clean(), validate(), store(), load()
# Hint: think about what `download()` should do when one ticker out of 50 fails. Should it crash? Should it silently drop the ticker? Should it record the failure and keep going? The right answer determines whether your pipeline is usable at scale.

In [ ]:
# TODO: Implement validate() — what metrics capture data quality?
# Think: completeness, stale prices, OHLC consistency, long gaps

# For storage, consider: when would you want wide format (tickers as columns) vs. long format (ticker as a row-level field)? Each serves different downstream use cases.
# TODO: Implement store() and load() with Parquet + metadata
# TODO: Test the full pipeline on 50 tickers

---
## ━━━ SOLUTION: Deliverable 1 ━━━

We'll build the `FinancialDataLoader` incrementally — one method at a time, with design commentary between each piece. This is a class with six responsibilities (init, download, clean, validate, store, load), and each one involves decisions that matter downstream. The monkey-patching pattern lets us discuss each method in isolation before wiring them together.

Let's start with the skeleton: what state does a data loader need to carry?

In [ ]:
class FinancialDataLoader:
    """
    Download, clean, validate, and store financial data.

    Parameters
    ----------
    tickers : list of str
        Stock ticker symbols to download.
    start_date, end_date : str
        Date range in YYYY-MM-DD format.
    cache_dir : Path or str
        Directory for storing downloaded data.
    ffill_limit : int
        Maximum consecutive days to forward-fill.
    """

    def __init__(self, tickers, start_date, end_date,
                 cache_dir=None, ffill_limit=5):
        self.tickers = tickers
        self.start_date = pd.Timestamp(start_date)
        self.end_date = pd.Timestamp(end_date)
        self.cache_dir = Path(cache_dir) if cache_dir else CACHE_DIR
        self.cache_dir.mkdir(exist_ok=True, parents=True)
        self.ffill_limit = ffill_limit
        self.data = None
        self.quality_scores = {}
        self.failed_tickers = []

Three pieces of state worth noting: `data` (the actual DataFrame), `quality_scores` (a dict of per-ticker metrics we'll populate in `validate()`), and `failed_tickers` (the ones yfinance couldn't deliver). The `ffill_limit` parameter is a design choice — forward-filling 1-2 missing days is standard practice (holidays, data vendor hiccups), but filling a 30-day gap would be fabricating data. Five days is a reasonable default: it covers a week of missing data but won't silently paper over a delisting.

Now the first real method: downloading. This is where most toy pipelines break at scale.

In [ ]:
def download(self):
    """Download OHLCV data, handling failures gracefully."""
    print(f"Downloading {len(self.tickers)} tickers...")
    try:
        self.data = yf.download(
            self.tickers, start=self.start_date,
            end=self.end_date, auto_adjust=False,
            progress=False)
        if len(self.tickers) == 1:
            self.data = pd.concat(
                [self.data], axis=1, keys=[self.tickers[0]]
            ).swaplevel(axis=1)
        print(f"Downloaded {self.data.shape[0]} rows")
    except Exception as e:
        print(f"Download failed: {e}")
        raise

    close = self.data["Close"]
    for t in self.tickers:
        if t in close.columns and close[t].isna().all():
            self.failed_tickers.append(t)
    if self.failed_tickers:
        print(f"Failed: {self.failed_tickers}")
    return self

FinancialDataLoader.download = download

Two critical design decisions here. First, `auto_adjust=False` — we want both raw Close and Adjusted Close so we can compute adjustment factors later. If you use `auto_adjust=True`, you lose the ability to detect splits and dividends programmatically. Second, the failure-detection loop: yfinance doesn't throw exceptions for individual ticker failures. It quietly returns all-NaN columns. The loop catches these and records them in `failed_tickers` so the rest of the pipeline knows which tickers to skip.

The single-ticker special case is a yfinance quirk — when you pass one ticker, the DataFrame structure changes (no MultiIndex). The `concat` + `swaplevel` normalizes it back. Small thing, but it would cause a crash at 1 AM when someone runs the pipeline for a single-stock analysis.

Next: cleaning. This is where we deal with the reality that financial calendars are messy.

In [ ]:
def clean(self):
    """Forward-fill gaps within limit, flag long gaps."""
    if self.data is None:
        raise ValueError("Must download() before clean()")
    print("Cleaning data...")

    us_bd = CustomBusinessDay(calendar=USFederalHolidayCalendar())
    expected_dates = pd.date_range(
        start=self.start_date, end=self.end_date, freq=us_bd
    )
    self.data = self.data.reindex(expected_dates)

    for column in self.data.columns:
        self.data[column] = self.data[column].ffill(
            limit=self.ffill_limit
        )
    print(f"Forward-filled gaps up to {self.ffill_limit} days")
    return self

FinancialDataLoader.clean = clean

The `reindex` call is doing important work. yfinance returns data on days the market was open — but "open" depends on which exchange, and sometimes there are half-days, early closes, or weather events. We generate the expected trading calendar using `USFederalHolidayCalendar` and reindex to it. Any days that yfinance missed become NaN, and we forward-fill up to the configured limit. Days that remain NaN after forward-filling are genuine long gaps — potential delistings, trading halts, or data vendor failures. Those stay NaN on purpose: they're signals, not noise.

Now the validation engine — the method that makes this a data *pipeline* rather than a data *downloader*. This one is the longest, so we'll build it in two parts: first the per-ticker metric computation, then the grading function.

In [ ]:
def validate(self):
    """Compute quality scores for each ticker."""
    if self.data is None:
        raise ValueError("Must download/clean before validate()")
    print("Validating data quality...")

    us_bd = CustomBusinessDay(calendar=USFederalHolidayCalendar())
    expected = pd.date_range(
        start=self.start_date, end=self.end_date, freq=us_bd)
    n_exp = len(expected)
    cols = {f: self.data[f] for f in
            ["Close", "Open", "High", "Low", "Volume"]}

    for ticker in self.tickers:
        if ticker in self.failed_tickers:
            self.quality_scores[ticker] = {
                "completeness": 0.0, "stale_days": 0,
                "ohlc_violations": 0, "long_gaps": 0,
                "grade": "F"}
            continue
        self.quality_scores[ticker] = self._score_ticker(
            ticker, cols, expected, n_exp)

    print(f"Scored {len(self.quality_scores)} tickers")
    return self

FinancialDataLoader.validate = validate

The `validate()` method delegates the per-ticker work to `_score_ticker()`. This keeps the main loop readable: for each ticker, either mark it as failed or compute its scores. The helper method does the heavy lifting — four quality dimensions, each catching a different failure mode.

In [ ]:
def _score_ticker(self, ticker, cols, expected, n_exp):
    """Compute quality metrics for a single ticker."""
    close_t = cols["Close"][ticker]
    n_actual = close_t.dropna().shape[0]
    completeness = n_actual / n_exp

    prices = close_t.dropna()
    stale_days = int(
        (prices.round(4) == prices.round(4).shift(1)).sum())

    idx = prices.index
    h, l = cols["High"][ticker][idx], cols["Low"][ticker][idx]
    o, c = cols["Open"][ticker][idx], close_t[idx]
    v = cols["Volume"][ticker][idx]
    ohlc_v = int(((h<o)|(h<c)).sum() + ((l>o)|(l>c)).sum()
                 + (v <= 0).sum())

    td = close_t.reindex(expected)
    m = td.isna()
    runs = m.astype(int).groupby((m != m.shift()).cumsum()).sum()
    long_gaps = int((runs > 5).sum())

    grade = self._compute_grade(
        completeness, stale_days, ohlc_v, long_gaps)
    return {"completeness": completeness,
            "stale_days": stale_days,
            "ohlc_violations": ohlc_v,
            "long_gaps": long_gaps, "grade": grade}

FinancialDataLoader._score_ticker = _score_ticker

Four quality dimensions, each catching a different failure mode. **Completeness** measures how many of the expected trading days actually have data — anything below 99% means the ticker has missing history (late IPO, early delisting, or vendor gap). **Stale prices** catch consecutive identical closes, which can mean illiquidity or a data feed that stopped updating. **OHLC violations** flag rows where High < Open or Low > Close — these should be impossible by definition, and their presence means something went wrong in aggregation. **Long gaps** (more than 5 consecutive missing days) flag potential delistings or trading halts that forward-fill can't responsibly cover.

The grading function translates these metrics into a letter grade that anyone at the fund can understand at a glance.

In [ ]:
def _compute_grade(self, completeness, stale_days,
                   ohlc_violations, long_gaps):
    """Assign A/B/C/D/F grade based on quality metrics."""
    if (completeness >= 0.99 and stale_days <= 20
            and ohlc_violations == 0 and long_gaps == 0):
        return "A"
    elif (completeness >= 0.95 and stale_days < 50
          and ohlc_violations < 5 and long_gaps == 0):
        return "B"
    elif (completeness >= 0.90 and stale_days < 100
          and ohlc_violations < 10):
        return "C"
    elif completeness >= 0.80:
        return "D"
    else:
        return "F"

FinancialDataLoader._compute_grade = _compute_grade

The grading thresholds encode real-world trade-offs. Grade A demands 99%+ completeness and zero OHLC violations — this is production-quality data you'd use for live trading signals. Grade B (95%+) is fine for backtesting research where small gaps won't materially change results. Grade C/D is "use with caution" territory — maybe acceptable for cross-sectional studies where one noisy ticker in 200 won't dominate the result. Grade F means the data is too broken to trust for anything.

These thresholds aren't academic — at a real fund, a risk analyst would define similar cutoffs and refuse to compute NAV on any ticker below grade B. The exact numbers are tunable; what matters is that the thresholds exist, are documented, and are enforced programmatically.

Now let's add the storage layer: saving to Parquet with metadata. We'll split this into two cells — one for the wide-format case (which is straightforward), and one for the long-format conversion plus the metadata save.

In [ ]:
def store(self, format="wide"):
    """Save data to Parquet with metadata."""
    if self.data is None or not self.quality_scores:
        raise ValueError(
            "Must download/clean/validate before store()"
        )
    print(f"Storing data in {format} format...")

    if format == "wide":
        data_path = self.cache_dir / "data_wide.parquet"
        self.data.to_parquet(data_path)
    elif format == "long":
        data_path = self.cache_dir / "data_long.parquet"
        self._store_long(data_path)
    else:
        raise ValueError(f"Unknown format: {format}")

    self._save_metadata(format, data_path)
    return self

FinancialDataLoader.store = store

The `store()` method delegates to two helpers: `_store_long()` for the wide-to-long conversion, and `_save_metadata()` for the sidecar JSON. This keeps the public API clean while handling the format-specific complexity behind the scenes.

In [ ]:
def _store_long(self, data_path):
    """Convert wide data to long format and save."""
    long_data = []
    for ticker in self.tickers:
        if ticker in self.failed_tickers:
            continue
        cols = {
            "date": self.data.index, "ticker": ticker,
            "open": self.data["Open"][ticker].values,
            "high": self.data["High"][ticker].values,
            "low": self.data["Low"][ticker].values,
            "close": self.data["Close"][ticker].values,
            "volume": self.data["Volume"][ticker].values,
            "adj_close": self.data["Adj Close"][ticker].values,
        }
        long_data.append(pd.DataFrame(cols))
    long_df = pd.concat(long_data, ignore_index=True)
    long_df.to_parquet(data_path, index=False)

FinancialDataLoader._store_long = _store_long

The long-format conversion iterates through tickers and stacks them vertically — each row becomes a single date-ticker observation. This is the natural shape for SQL-style queries and for Polars, which expects tidy data. The wide format is better for pandas cross-sectional operations (correlation matrices, relative momentum).

Now the metadata sidecar — the provenance record that travels with the data.

In [ ]:
def _save_metadata(self, format, data_path):
    """Write sidecar JSON with provenance info."""
    metadata = {
        "tickers": self.tickers,
        "start_date": str(self.start_date.date()),
        "end_date": str(self.end_date.date()),
        "download_date": str(pd.Timestamp.now()),
        "source": "yfinance",
        "failed_tickers": self.failed_tickers,
        "quality_scores": self.quality_scores,
        "format": format,
    }
    meta_path = self.cache_dir / f"metadata_{format}.json"
    with open(meta_path, "w") as f:
        json.dump(metadata, f, indent=2)
    print(f"Saved to {data_path}")
    print(f"Metadata: {meta_path}")

FinancialDataLoader._save_metadata = _save_metadata

Two format options, each serving a different use case. **Wide format** (tickers as column levels) is what you want for time-series analysis — slicing `data["Close"]` gives you a DataFrame where each column is a ticker, perfect for cross-sectional computations like correlation matrices or relative momentum. **Long format** (ticker as a regular column) is what databases and Polars prefer — it's the natural shape for SQL-style queries like "give me all rows where ticker='AAPL' and date > '2020-01-01'."

The sidecar JSON metadata is a small but important detail. When someone loads this data in three months, they'll want to know: when was it downloaded? What source? Which tickers failed? What were the quality scores? Embedding this in the file system alongside the data means the provenance travels with the data, not in someone's memory.

Last method: loading with flexible filtering. We'll split this into the core load and the filtering logic.

In [ ]:
def load(self, tickers=None, start_date=None,
         end_date=None, format="wide", min_grade=None):
    """Load data from Parquet with optional filtering."""
    data_path = self.cache_dir / f"data_{format}.parquet"
    meta_path = self.cache_dir / f"metadata_{format}.json"
    if not data_path.exists():
        raise FileNotFoundError(f"No data at {data_path}")

    with open(meta_path) as f:
        meta = json.load(f)
    self.quality_scores = meta["quality_scores"]
    self.failed_tickers = meta["failed_tickers"]

    data = pd.read_parquet(data_path)
    data = self._apply_filters(
        data, tickers, start_date, end_date,
        min_grade, format
    )
    self.data = data
    print(f"Loaded {len(data)} rows from {data_path}")
    return data

FinancialDataLoader.load = load

The `load()` method reads metadata first (restoring quality scores and failed-ticker info), then delegates all filtering to `_apply_filters()`. This separation makes it easy to add new filter dimensions later — by date, by sector, by minimum volume — without touching the core load logic.

In [ ]:
def _apply_filters(self, data, tickers, start_date,
                   end_date, min_grade, format):
    """Apply ticker, date, and quality filters."""
    def _by_tickers(df, tk_list):
        if format == "wide":
            ok = [t for t in tk_list if t in df["Close"].columns]
            return df.loc[:, (slice(None), ok)]
        return df[df["ticker"].isin(tk_list)]

    if tickers:
        data = _by_tickers(data, tickers)
    for bound, op in [(start_date, "ge"), (end_date, "le")]:
        if bound:
            ts = pd.Timestamp(bound)
            key = data.index if format == "wide" else data["date"]
            data = data[key >= ts] if op == "ge" else data[key <= ts]
    if min_grade:
        rank = {"A": 5, "B": 4, "C": 3, "D": 2, "F": 1}
        good = [t for t, s in self.quality_scores.items()
                if rank.get(s["grade"], 0) >= rank.get(min_grade, 0)]
        data = _by_tickers(data, good)
    return data

FinancialDataLoader._apply_filters = _apply_filters

The filtering supports three dimensions: tickers, date range, and quality grade. The quality-grade filter is the one that matters most in practice. Imagine a researcher says: "Give me all grade-A data for the last 5 years." One line: `loader.load(min_grade="A", start_date="2020-01-01")`. No manual ticker lists, no guessing which stocks are clean. The quality metadata, computed once at validation time, serves as a permanent label.

The class is complete. Let's run the full pipeline on 50 tickers and see what happens.

In [ ]:
test_tickers = [
    "AAPL", "MSFT", "GOOGL", "AMZN", "META", "NVDA", "TSLA",
    "NFLX", "JPM", "BAC", "WFC", "GS", "MS", "C", "BLK",
    "JNJ", "UNH", "PFE", "LLY", "ABBV", "TMO",
    "WMT", "HD", "PG", "KO", "PEP", "COST", "NKE",
    "BA", "CAT", "GE", "MMM", "HON",
    "XOM", "CVX", "COP", "SLB",
    "SPY", "QQQ", "IWM", "DIA", "EEM", "TLT", "GLD",
    "AMD", "INTC", "PYPL", "ORCL", "ZM", "UBER", "V"
]

loader = FinancialDataLoader(
    tickers=test_tickers,
    start_date="2010-01-01",
    end_date="2025-01-01",
    ffill_limit=5
)

loader.download().clean().validate().store(format="wide")

The chained `.download().clean().validate().store()` call is the intended API — each method returns `self`, so you can run the full pipeline in one line. If any step fails, the exception includes a clear message about what prerequisite was missing.

Look at the output above. Some tickers may have failed — this is expected and exactly what the pipeline is designed to handle. yfinance is a free API with rate limits and occasional schema changes. A production pipeline at a real fund would have retry logic with exponential backoff, but the principle is the same: download what you can, record what failed, move on.

Let's see the quality score distribution.

In [ ]:
quality_df = pd.DataFrame(loader.quality_scores).T
quality_df = quality_df.sort_values("grade")

print(f"Total tickers: {len(test_tickers)}")
print(f"Failed downloads: {len(loader.failed_tickers)}")
print(f"\nGrade distribution:")
print(quality_df["grade"].value_counts().sort_index())
print(f"\nBottom 10 by quality:")
print(quality_df.tail(10).to_string())

Notice the pattern: the mega-cap liquid names (AAPL, MSFT, SPY) almost certainly got grade A — near-perfect completeness, no OHLC violations, minimal stale prices. But scroll down to the tickers with shorter histories (UBER IPO'd in 2019, ZM IPO'd in 2019, META was FB until 2022) and you'll start seeing lower completeness scores. They don't have data back to 2010 because they didn't exist yet. The grading system correctly catches this: a ticker with 5 years of history out of a 15-year window gets dinged on completeness.

This is exactly the kind of thing that bites you silently in a backtest. If you compute a 15-year Sharpe ratio on a ticker that only has 5 years of data, you're not measuring what you think you're measuring.

Now let's verify the round-trip: does what we save come back unchanged?

In [ ]:
original_data = loader.data.copy()
loader.store(format="wide")
loader.store(format="long")
loaded_wide = loader.load(format="wide")

close_orig = original_data["Close"].values
close_load = loaded_wide["Close"].values
max_diff = np.nanmax(np.abs(close_orig - close_load))

print(f"Original shape: {original_data.shape}")
print(f"Loaded shape:   {loaded_wide.shape}")
print(f"Max abs difference: {max_diff:.10f}")

That max difference should be zero or negligibly close to machine epsilon. This is one of Parquet's strengths over CSV — it stores floats in binary, so there's no string-to-float conversion that could introduce rounding artifacts. With CSV, a price of 192.36999999999998 might get truncated during writing and come back as 192.37. Over millions of rows, those micro-differences can compound into visible return discrepancies. Parquet avoids the problem entirely.

Final check: does filtering work correctly?

In [ ]:
subset = loader.load(
    tickers=["AAPL", "TSLA", "SPY"], format="wide"
)
print(f"3-ticker filter: {subset['Close'].columns.tolist()}")

recent = loader.load(
    start_date="2020-01-01", end_date="2022-12-31",
    format="wide"
)
print(f"Date filter (2020-2022): {len(recent)} rows")

hq = loader.load(min_grade="A", format="wide")
n_hq = (hq["Close"].shape[1]
        if "Close" in hq.columns.get_level_values(0) else 0)
print(f"Grade-A tickers: {n_hq}")

All three filters work as expected. The ticker filter returns exactly the 3 requested names. The date filter slices down to 3 years of history. And the grade filter — the most powerful one — returns only the tickers that passed production quality standards. In a real workflow, a researcher would always start with `min_grade="B"` at minimum. Using grade-D data for a backtest is like training an ML model on a dataset with 20% corrupted labels: your results might look fine on average, but you'd never trust them for a real decision.

**The aha moment for Deliverable 1:** Building a data pipeline for 5 stocks is trivially easy — a single `yf.download()` call and you're done. Scaling to 50 reveals an entirely different class of problems: tickers that changed symbols (Facebook to META), IPOs that create incomplete histories (UBER, ZM), ETFs with slightly different trading calendars (EEM tracks emerging markets with occasional holidays the US market ignores), and the infamous BRK-B ticker that yfinance wants as `BRK-B` not `BRK.B`. Each edge case is minor in isolation. Together, they're the reason quant data engineering is a full-time job at every serious fund.

---

## Deliverable 2: Data Quality Report

The `FinancialDataLoader` computes quality scores. Now turn those scores into a report that a portfolio manager could read and act on. Per-ticker metrics with letter grades. A universe-level summary. A call-out of the problematic names.

The goal isn't a pretty table — it's an answer to the question every PM will ask before trusting your pipeline: "How do I know this data is good?"

**Requirements:**
- Per-ticker metrics: completeness, stale days, OHLC violations, long gaps, letter grade
- Universe summary: how many tickers pass production quality, grade distribution
- At least one ticker should receive a grade below A (if everything gets an A, your standards are too low)

In [ ]:
# TODO: Build a per-ticker report table from loader.quality_scores
# TODO: Compute universe-level summary (grade distribution)

Think about what "production quality" means. What completeness threshold separates data you'd trust for a live trading signal from data you'd only use for exploratory research?

In [ ]:
# TODO: Highlight problematic tickers
# TODO: Visualize completeness distribution

---
## ━━━ SOLUTION: Deliverable 2 ━━━

We already computed quality scores in Deliverable 1. The report just needs to present them in a way that's immediately actionable — no digging through raw dictionaries. A portfolio manager should be able to glance at this and know which tickers to trust.

In [ ]:
quality_scores = loader.quality_scores

report_rows = []
for ticker, m in quality_scores.items():
    report_rows.append({
        "Ticker": ticker,
        "Completeness": f"{m['completeness']:.1%}",
        "Stale Days": m["stale_days"],
        "OHLC Violations": m["ohlc_violations"],
        "Long Gaps (>5d)": m["long_gaps"],
        "Grade": m["grade"]
    })

per_ticker_report = pd.DataFrame(report_rows)
per_ticker_report = per_ticker_report.sort_values("Grade")
per_ticker_report

Scan the Grade column. The A-rated tickers are your workhorses — use them without hesitation for backtesting, model training, and risk analysis. The B-rated ones are acceptable for most research but worth a manual check before putting them in production. Anything below B deserves investigation: why is completeness low? Is it a late IPO (legitimate) or a data gap (concerning)?

The grading criteria are documented and reproducible:
- **A:** Completeness >= 99%, stale days <= 20, zero OHLC violations, zero long gaps
- **B:** Completeness >= 95%, stale days < 50, OHLC violations < 5, no long gaps
- **C:** Completeness >= 90%, stale days < 100, OHLC violations < 10
- **D:** Completeness >= 80%
- **F:** Below all thresholds

Now let's aggregate to the universe level.

In [ ]:
grade_counts = per_ticker_report["Grade"].value_counts()
grade_counts = grade_counts.sort_index()
total_tickers = len(per_ticker_report)

print(f"Total tickers analyzed: {total_tickers}")
print(f"Failed downloads: {len(loader.failed_tickers)}\n")
print("Grade Distribution:")
for grade in ["A", "B", "C", "D", "F"]:
    count = grade_counts.get(grade, 0)
    pct = count / total_tickers * 100
    print(f"  {grade}: {count:3d} ({pct:5.1f}%)")

prod = grade_counts.get("A", 0) + grade_counts.get("B", 0)
print(f"\nProduction-quality (A or B): "
      f"{prod} ({prod/total_tickers*100:.1f}%)")

The universe summary tells you the overall health of your data pipeline in two numbers: total tickers and the percentage that pass production quality. If you're running a fund with 50 names and only 35 pass grade B or better, you either need a better data vendor for the other 15, or you need to exclude them from analysis. Either way, you need to know — and this report gives you that answer automatically.

Let's call out the worst offenders.

In [ ]:
problematic = per_ticker_report[
    per_ticker_report["Grade"].isin(["C", "D", "F"])
]

if len(problematic) > 0:
    print("Tickers requiring investigation (Grade C or below):")
    print(problematic.to_string(index=False))
else:
    print("All tickers scored B or above.")

Any tickers flagged above deserve a manual look. Low completeness on UBER or ZM is explained by their recent IPO dates (2019) — that's legitimate, not a data error. But low completeness on a stock that's been trading since 2005 would be a red flag pointing to a data vendor gap. The report surfaces the issue; the human investigates.

Let's visualize the completeness distribution to see the overall picture.

In [ ]:
completeness_vals = [
    m["completeness"] for m in quality_scores.values()
]

fig, ax = plt.subplots(figsize=(10, 5))
ax.hist(completeness_vals, bins=20, edgecolor="black",
        alpha=0.7)
ax.axvline(0.99, color="green", ls="--",
           label="Production (99%)")
ax.axvline(0.90, color="orange", ls="--",
           label="Minimum (90%)")
ax.set_xlabel("Completeness Ratio")
ax.set_ylabel("Number of Tickers")
ax.set_title("Data Completeness Distribution Across Universe")
ax.legend()
plt.tight_layout()
plt.show()

**The aha moment for Deliverable 2:** The histogram makes it visceral. Most tickers cluster at or near 100% completeness — these are the household names with deep liquidity and robust vendor coverage. But there's a tail on the left: tickers with 30-70% completeness. These are typically recent IPOs being asked for 15 years of history. The real danger, though, isn't the tickers you know are incomplete — it's the ones at 95% completeness, where 5% of the trading days are silently missing. A 5% gap spread over 15 years means roughly one week of data missing per year. If one of those missing weeks happens to be March 2020 (COVID crash), your volatility estimates are materially understated and your max-drawdown figure is a fiction. Automated quality checks aren't optional — they're the difference between a pipeline you can trust and one that lies to you with a straight face.

---

## Deliverable 3: Storage Format Benchmark

The lecture mentioned that Parquet beats CSV and Polars beats pandas. Now prove it — with numbers, at multiple scales. Save the same dataset as CSV and Parquet, read it back with pandas and Polars, and measure everything: file sizes, write times, read times.

The interesting part isn't that Parquet wins (it will). It's *how the gap scales*. At 10 tickers, the difference might barely matter. At 50, it's meaningful. Extrapolate to the 500-ticker universe a real fund uses, and you'll understand why quant teams don't store data in CSV.

**Requirements:**
- Test all 4 combinations: CSV+pandas, CSV+Polars, Parquet+pandas, Parquet+Polars
- Test at 3 scales (10, 25, all tickers)
- Report file sizes, read times, and speedup ratios
- Verify data integrity across all combinations

In [ ]:
# TODO: Load the full dataset and create subsets at 3 scales
# TODO: Benchmark all 4 format+library combinations

Think about what "scales" means in practice. A 10-ticker subset might fit entirely in L3 cache. A 500-ticker dataset probably won't. The performance characteristics can change dramatically when data spills from cache to main memory to disk.

In [ ]:
# TODO: Display results table and plot scaling behavior
# TODO: Verify data integrity across formats

---
## ━━━ SOLUTION: Deliverable 3 ━━━

A benchmark at a single scale is barely a benchmark — it tells you what's faster *right now* but not what happens when your data grows. We'll test at three scales (10, 25, and all ~50 tickers) to see whether the performance gap stays constant, widens, or narrows. The answer matters for capacity planning: if Polars+Parquet is only 20% faster at 50 tickers but 10x faster at 500, you want to know that before your data outgrows your tools.

In [ ]:
full_data = loader.load(format="wide")
all_tickers = full_data["Close"].columns.tolist()

print(f"Full dataset: {full_data.shape[0]} rows x "
      f"{full_data.shape[1]} columns")
print(f"Available tickers: {len(all_tickers)}")
mem_mb = full_data.memory_usage(deep=True).sum() / 1e6
print(f"Memory usage: {mem_mb:.1f} MB")

bench_dir = CACHE_DIR / "benchmark"
bench_dir.mkdir(exist_ok=True, parents=True)

The memory usage number is worth noting. This is a ~50-ticker, 15-year daily dataset — a tiny fraction of what a real fund works with. If this already occupies meaningful memory, imagine a 5,000-ticker minute-bar dataset. Format and library choices that seem like micro-optimizations at our scale become existential at production scale.

Now the benchmark. We write each subset once (CSV and Parquet), then read it back with all four library+format combinations. The key metric is the speedup relative to the slowest combination (pandas+CSV).

In [ ]:
SCALES = [10, 25, len(all_tickers)]
all_results = []

def _time_read(fn, *a, **kw):
    t0 = time.perf_counter(); fn(*a, **kw); return time.perf_counter() - t0

for n in SCALES:
    sub = full_data.loc[:, (slice(None), all_tickers[:n])]
    csv_p, pq_p = bench_dir / f"bench_{n}.csv", bench_dir / f"bench_{n}.parquet"
    sub.to_csv(csv_p); sub.to_parquet(pq_p)
    csv_mb, pq_mb = csv_p.stat().st_size / 1e6, pq_p.stat().st_size / 1e6
    pd_csv = _time_read(pd.read_csv, csv_p, index_col=0,
                        parse_dates=True, header=[0, 1])
    pd_pq = _time_read(pd.read_parquet, pq_p)
    pl_csv = _time_read(pl.read_csv, csv_p)
    pl_pq = _time_read(pl.read_parquet, pq_p)
    for lib, fmt, rd in [
        ("pandas", "CSV", pd_csv), ("pandas", "Parquet", pd_pq),
        ("Polars", "CSV", pl_csv), ("Polars", "Parquet", pl_pq),
    ]:
        all_results.append({"Tickers": n, "Library": lib,
            "Format": fmt, "Read (s)": rd, "Speedup": pd_csv / rd})
    print(f"{n} tickers: CSV={csv_mb:.2f}MB, "
          f"Parquet={pq_mb:.2f}MB ({csv_mb/pq_mb:.1f}x)")

results_df = pd.DataFrame(all_results)

Look at those file size ratios. Parquet should be 3-5x smaller than CSV at every scale. That compression comes from two sources: Parquet stores numbers in binary (no string conversion overhead), and it uses columnar compression that exploits the fact that a column of float64 prices has much less entropy than the same numbers as ASCII text. At our toy scale, the absolute savings are a few megabytes. At production scale — think 5,000 tickers with minute bars for 20 years — the difference is hundreds of gigabytes.

Now let's look at the read-time comparison in table form.

In [ ]:
for n in SCALES:
    print(f"\n--- {n} tickers ---")
    s = results_df[results_df["Tickers"] == n]
    cols = ["Library", "Format", "Read (s)", "Speedup"]
    print(s[cols].to_string(index=False))

The "Speedup" column is the one to watch. At 10 tickers, Polars+Parquet might only be 2-3x faster — noticeable but not life-changing. At 50 tickers, the gap should widen. The reason is architectural: CSV reading is fundamentally serial (parse each character, detect delimiters, convert types), while Parquet reading is parallelizable (the columnar layout lets you read independent column chunks simultaneously). Polars exploits this parallelism aggressively; pandas is more conservative.

Let's visualize the scaling behavior.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

for (lib, fmt), grp in results_df.groupby(["Library", "Format"]):
    axes[0].plot(grp["Tickers"], grp["Read (s)"],
                 marker="o", label=f"{lib}+{fmt}", lw=1.5)
axes[0].set_xlabel("Number of Tickers")
axes[0].set_ylabel("Read Time (s)")
axes[0].set_title("Read Time vs. Dataset Scale")
axes[0].legend(fontsize=9)

for (lib, fmt), grp in results_df.groupby(["Library", "Format"]):
    if not (lib == "pandas" and fmt == "CSV"):
        axes[1].plot(grp["Tickers"], grp["Speedup"],
                     marker="o", label=f"{lib}+{fmt}", lw=1.5)
axes[1].set_xlabel("Number of Tickers")
axes[1].set_ylabel("Speedup vs pandas+CSV")
axes[1].set_title("Speedup Scaling")
axes[1].legend(fontsize=9)
axes[1].axhline(1.0, color="gray", ls="--", lw=0.5)

plt.tight_layout()
plt.show()

The left panel shows raw read times. If the lines are diverging as ticker count increases, that's the scaling story: the gap between the fastest combination (Polars+Parquet) and the slowest (pandas+CSV) grows with data size. The right panel normalizes everything to the pandas+CSV baseline, making the speedup factor explicit.

This is why the industry is migrating. At 50 tickers the time savings might be seconds. At 5,000 tickers with intraday data, those seconds become minutes — and when a researcher iterates on a strategy 50 times a day, minutes become hours.

One more thing: let's verify that switching formats doesn't corrupt the data.

In [ ]:
csv_p = bench_dir / f"bench_{len(all_tickers)}.csv"
pq_p = bench_dir / f"bench_{len(all_tickers)}.parquet"

df_csv = pd.read_csv(
    csv_p, index_col=0, parse_dates=True, header=[0, 1]
)
df_pq = pd.read_parquet(pq_p)

c_csv = df_csv.iloc[
    :, df_csv.columns.get_level_values(0) == "Close"
].values
c_pq = df_pq.iloc[
    :, df_pq.columns.get_level_values(0) == "Close"
].values

diff = np.nanmax(np.abs(c_csv - c_pq))
print(f"Max diff (CSV vs Parquet Close): {diff:.10f}")

**The aha moment for Deliverable 3:** The size and speed advantages of Parquet over CSV are well-known in the abstract — everyone has heard "Parquet is faster." What this multi-scale benchmark reveals is that the advantage is *super-linear* with data size. At 10 tickers, Parquet might be 3x smaller and 2x faster to read. At 50, it's 4x smaller and 4x faster. Extrapolate to the 10,000-instrument universe at a large quant fund with 20 years of minute bars, and the difference is between a 2 TB storage bill and a 500 GB one, between a 10-minute data load and a 60-second one. The format choice that seems like a minor engineering detail at toy scale becomes a defining infrastructure decision at production scale. This is why every quant fund built in the last 5 years uses Parquet as its default storage format — and why teams still on CSV are actively migrating.

---

## Deliverable 4: Return Statistics Summary (Bridge to Week 2)

This is the bridge. Compute basic return statistics for 10 representative tickers spanning different sectors and volatility profiles. The numbers you produce here will motivate everything in Week 2: stylized facts, fat tails, volatility clustering, non-stationarity.

The key question: are returns normally distributed? Your ML training has probably assumed yes (or at least close enough). The numbers will say otherwise — and the magnitude of the violation will surprise you.

**Requirements:**
- 10 tickers spanning sectors: tech, finance, healthcare, energy, consumer, industrials, plus ETFs
- Compute: mean daily return, annualized volatility, skewness, kurtosis, min/max, and the number of >3-sigma days
- Compare the actual tail behavior to what a Gaussian would predict
- Present in a clean summary table

In [ ]:
# TODO: Select 10 representative tickers across sectors
# TODO: Compute daily log returns

Choose tickers that span the volatility spectrum: at least one defensive name (utility or consumer staple), one high-volatility name (growth tech), and one non-equity (gold ETF or bond ETF). If the normality violations show up across all of these, the conclusion is structural, not anecdotal.

In [ ]:
# TODO: Compute statistics and build summary table
# TODO: Visualize distributions vs. Gaussian

---
## ━━━ SOLUTION: Deliverable 4 ━━━

We want diversity in our 10 tickers — not 10 tech stocks that all move together. The selection below spans sectors, volatility profiles, and asset types: defensive healthcare (JNJ), volatile tech (TSLA), cyclical energy (XOM), the broad market benchmark (SPY), and a non-equity asset (GLD). If the normality violations appear across all of these, we'll know it's a structural property of financial returns, not a quirk of one sector.

In [ ]:
STAT_TICKERS = [
    "AAPL",   # Tech, large-cap
    "TSLA",   # Tech/auto, high volatility
    "JPM",    # Financials
    "JNJ",    # Healthcare, defensive
    "XOM",    # Energy
    "WMT",    # Consumer staples, defensive
    "NFLX",   # Media/entertainment
    "BA",     # Industrials
    "SPY",    # S&P 500 ETF (market benchmark)
    "GLD"     # Gold ETF (alternative asset)
]

stat_data = loader.load(tickers=STAT_TICKERS, format="wide")
close = stat_data["Adj Close"]
returns = np.log(close / close.shift(1)).dropna()

print(f"Computing stats for {len(STAT_TICKERS)} tickers")
print(f"Date range: {returns.index[0].date()} to "
      f"{returns.index[-1].date()}")
print(f"Observations per ticker: ~{len(returns)}")

We're using log returns here, not simple returns. For statistical analysis — especially when testing for normality — log returns are the standard choice because they're additive across time and approximately symmetric for small moves. Simple returns would be the right choice if we were computing dollar P&L. The distinction matters: a stock that goes up 10% and then down 10% has a simple return of -1% (the compounding trap from the lecture), but log returns of +9.53% and -10.54% that honestly add up to a net negative.

Now let's compute the full statistics panel.

In [ ]:
stats_rows = []
for ticker in STAT_TICKERS:
    ret = returns[ticker].dropna()
    std_d = ret.std()
    extreme = int((ret.abs() > 3 * std_d).sum())
    ext_pct = extreme / len(ret)

    stats_rows.append({
        "Ticker": ticker,
        "Mean (daily)": ret.mean(),
        "Vol (annual)": std_d * np.sqrt(252),
        "Skewness": scipy_stats.skew(ret),
        "Kurtosis": scipy_stats.kurtosis(ret) + 3,
        "Min": ret.min(), "Max": ret.max(),
        "> 3s days": extreme,
        "> 3s %": ext_pct,
        "vs Normal": ext_pct / 0.0027,
    })

stats_df = pd.DataFrame(stats_rows)

A quick note on the kurtosis computation: `scipy.stats.kurtosis` returns *excess* kurtosis (Gaussian = 0). We add 3 to convert to absolute kurtosis (Gaussian = 3), which is the more common convention in finance. When someone at a fund says "TSLA has kurtosis 20," they mean absolute kurtosis. The "vs Normal" column divides the actual percentage of extreme days by the Gaussian expectation of 0.27% — a ratio of 3x means you're seeing three times more tail events than a Gaussian model would predict.

Let's format this into a readable table.

In [ ]:
disp = stats_df.copy()
disp["Mean (daily)"] = disp["Mean (daily)"].map("{:.4f}".format)
disp["Vol (annual)"] = disp["Vol (annual)"].map("{:.1%}".format)
disp["Skewness"] = disp["Skewness"].map("{:+.2f}".format)
disp["Kurtosis"] = disp["Kurtosis"].map("{:.1f}".format)
disp["Min"] = disp["Min"].map("{:.1%}".format)
disp["Max"] = disp["Max"].map("{:.1%}".format)
disp["> 3s %"] = disp["> 3s %"].map("{:.2%}".format)
disp["vs Normal"] = disp["vs Normal"].map("{:.1f}x".format)

print("DAILY RETURN STATISTICS (2010-2025)")
print("=" * 100)
print(disp.to_string(index=False))

Read this table row by row and let the numbers sink in. Every single ticker has kurtosis well above 3. If returns were Gaussian, kurtosis would be 3.0 for all of them. Instead, you're seeing values ranging from maybe 4-5 for the defensives (JNJ, WMT) up to 15-25+ for the volatile names (TSLA, BA). The "vs Normal" column is the punchline: for most tickers, extreme events (beyond 3 standard deviations) happen 2-5x more often than a Gaussian model predicts. For the most volatile names, it can be 5-10x more frequent.

The skewness column tells a parallel story: most tickers have negative skewness, meaning the left tail (crashes) is fatter than the right tail (rallies). Losses are not just more frequent than Gaussian models predict — they're also larger than the corresponding gains. This asymmetry is one reason options markets price puts more expensively than calls.

Let's visualize the distributions against their Gaussian counterparts.

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 4))
plot_tickers = ["JNJ", "SPY", "TSLA"]

for ax, ticker in zip(axes, plot_tickers):
    ret = returns[ticker].dropna()
    ax.hist(ret, bins=50, density=True, alpha=0.7,
            label="Empirical", edgecolor="black")
    mu, sig = ret.mean(), ret.std()
    x = np.linspace(ret.min(), ret.max(), 100)
    ax.plot(x, scipy_stats.norm.pdf(x, mu, sig),
            "r-", lw=2, label="Gaussian")
    ax.set_title(f"{ticker} Daily Returns")
    ax.set_xlabel("Return")
    ax.set_ylabel("Density")
    ax.legend(fontsize=8)

plt.tight_layout()
plt.show()

Three tickers, three volatility regimes, one universal pattern. JNJ is the quietest of the three — a defensive healthcare stock that barely moves most days. Even so, its empirical distribution has a sharper peak and fatter tails than the Gaussian overlay. SPY, the broad market, shows the same pattern more clearly: the red Gaussian curve underestimates the peak and wildly underestimates the tails. TSLA is where it gets dramatic — the empirical distribution has tails stretching to daily moves of 15-20%, events that a Gaussian model would assign probability approximately zero.

The visual makes the statistics table tangible. A kurtosis number is abstract. Seeing the Gaussian curve hugging zero while the empirical histogram extends far beyond it in both directions — that's what fat tails actually look like.

**The aha moment for Deliverable 4:** Compare JNJ and TSLA side by side. JNJ's kurtosis might be around 5-6 — elevated above Gaussian, but not terrifyingly so. TSLA's could be 15-25+ — a factor of 4-5x higher. They're both in the S&P 500, both trade billions of dollars a day, but their tail risk profiles are in different universes. If your risk model uses a single distributional assumption for the whole portfolio (as many standard VaR models do), it will dramatically understate risk for the TSLA-like names and marginally overstate it for the JNJ-like names. This heterogeneity isn't a nuisance — it's a fundamental feature of financial returns that demands asset-specific distributional modeling. Week 2 will give you the tools: GARCH for time-varying volatility, EVT for tail modeling, and stylized facts that apply universally despite this cross-sectional variation.

---

## What You Built Tonight

Four deliverables, one pipeline, and a collection of numbers that should make you permanently skeptical of any dataset that arrives without quality documentation.

- **The `FinancialDataLoader` handles 50+ tickers** without crashing on individual failures. It downloads, cleans, validates, stores to Parquet, and loads with flexible filtering — the complete lifecycle that every production data pipeline needs, even though most toy projects skip everything after the download step.

- **Data quality varies dramatically across tickers**, even within a curated universe of well-known US stocks. Recent IPOs, corporate restructurings, and ticker changes all create gaps that a naive pipeline would silently ignore. The quality grading system (A through F) makes reliability visible at a glance — and reveals that "download from yfinance" is not the same as "have clean data."

- **Parquet beats CSV by 3-5x on file size and the gap in read performance widens with scale.** At 10 tickers, the format choice barely matters. At 50, it's meaningful. Extrapolate to the thousands of instruments a real fund tracks, and the format choice becomes an infrastructure-defining decision. Polars adds another multiplier on top.

- **Every ticker in the universe has fat tails** — kurtosis well above the Gaussian value of 3, extreme events occurring 2-10x more often than normal models predict, and negative skewness meaning crashes are larger than rallies. This isn't a property of volatile stocks — even defensive names like JNJ and the broad market index SPY violate normality.

- **The kurtosis gap between defensive and volatile stocks** (JNJ at ~5 vs. TSLA at ~20+) means a one-size-fits-all distributional assumption is dangerously wrong. Asset-specific modeling isn't a luxury — it's a necessity.

- **Building for 5 tickers is trivially easy; building for 50 reveals an entirely different class of problems.** Ticker changes, short histories, vendor quirks, calendar mismatches — these are the reasons quant data engineering is a full-time job, not a pre-processing afterthought.

The pipeline you built tonight is the foundation for everything that follows. You'll load data from it in Week 2 (stylized facts), Week 3 (feature engineering), Week 4 (portfolio construction), and beyond. Next week, we dig into those kurtosis numbers — what causes them, what they break, and what to do about them.